In [4]:
# -*- coding: utf-8 -*-

import numpy as np
import csv
from scipy.sparse import coo_matrix


In [3]:
all_cut_word = 'E:\\PycharmProjects\\Demo0410\\work_data_liziqi\\all_cut_word.txt'

word = []  #这个列表将包含处理数据所有的词，不分文档。
f = open(all_cut_word,'r',encoding='utf-8')
line = f.readline()  #这里.readline()是读取下一行，如第一次则读取第一行，再读一次则读取第2行。.readlines()是将所有行读入到一个列表中。
while line:
    #print line
    line = line.replace("\n", "") #取消换行符。
    line = line.strip('\n')   #取消换行符前面或后面的空格。
    for n in line.split(' '):  #将所读的读用空格分开，并分别读取每个词，记为n。
        #print n
        if n not in word:  #如果读取的词n没有在word列表中，则将其添加到列表，否则不处理。此步主要是将不重复的词全部加到一个列表。
            word.append(n)
    line = f.readline() #读取下一行。
f.close()
#print(len(word)) #全部不重复的词的总数。经过去重后共得到6538个词。


FileNotFoundError: [Errno 2] No such file or directory: 'E:\\PycharmProjects\\Demo0410\\work_data_liziqi\\all_cut_word.txt'

In [16]:
a = np.zeros([2,3])  #先创建一个各元素皆为0的2*3矩阵（2行3列的空矩阵），以备后面添加数据。
print(a)


[[0. 0. 0.]
 [0. 0. 0.]]


In [18]:
word_vector = np.zeros([len(word),len(word)], dtype='float16')  #创建一个列和行总数都等于单词总数的矩阵。
print(word_vector.shape)

(6538, 6538)


In [20]:
f = open(all_cut_word, encoding='utf-8')  #再次读取文件。
line = f.readline()           #再次读取第一行。
while line:
    line = line.replace("\n", "") #再次过滤换行符。
    line = line.strip('\n') #再次过滤换行符前后的空格。
    nums = line.split(' ') #再次用空格分割。此时nums为列表，列表元素为一个一个的词。

    #循环遍历关键词所在位置 设置word_vector计数
    i = 0
    j = 0
    while i<len(nums):  #循环的次数为每行（每条评论）的词数。这个循环是处理每一行。
        j = i + 1  #第一次执行的时候i=0,j=1。
        w1 = nums[i]           #w1为此行的第一词。
        while j<len(nums):  #循环的次数仍然为每行（每条评论）的词数。
            w2 = nums[j]       #此时i=0,j=1，则w2为此行的第二个单词。
            #从word数组中找到单词对应的下标
            k = 0
            n1 = 0  #n1为行。
            while k<len(word):
                if w1==word[k]:  #第一次执行时，w1为第一个词。意思是，如果是这个词本身，就退出此循环，执行下一步。
                    n1 = k
                    break
                k = k +1  #如果不是这个词本身，那就执行这句。
            #寻找第二个关键字位置
            k = 0
            n2 = 0  #n2为列。
            while k<len(word):
                if w2==word[k]:
                    n2 = k
                    break
                k = k +1
            #重点: 词频矩阵赋值 只计算上三角
            if n1<=n2:
                word_vector[n1][n2] = word_vector[n1][n2] + 1
            else:
                word_vector[n2][n1] = word_vector[n2][n1] + 1
            #print n1, n2, w1, w2
            j = j + 1
        i = i + 1
    #读取新内容
    line = f.readline()
f.close()

In [27]:
res = open("E:\\PycharmProjects\\Demo0410\\work_data_liziqi\\word_word_weight.txt", "a+", encoding='utf-8')
i = 0
while i<len(word):
    w1 = word[i]
    j = 0
    while j<len(word):
        w2 = word[j]
        #判断两个词是否共现 共现&词频不为0的写入文件
        if word_vector[i][j]>0:
            #print w1 +" " + w2 + " "+ str(int(word_vector[i][j]))
            res.write(w1 +" " + w2 + " "+ str(int(word_vector[i][j]))  +  "\n")
        j = j + 1
    i = i + 1
res.close()

In [28]:
c = open("E:\\PycharmProjects\\Demo0410\\work_data_liziqi\\word-word-weight.csv","w", encoding='utf-8', newline='')    #解决空行
#c.write(codecs.BOM_UTF8)                                 #防止乱码
writer = csv.writer(c)                                    #写入对象
writer.writerow(['Word1', 'Word2', 'Weight'])

i = 0
while i<len(word):
    w1 = word[i]
    j = 0
    while j<len(word):
        w2 = word[j]
        #判断两个词是否共现 共现词频不为0的写入文件
        if word_vector[i][j]>0:
            #写入文件
            templist = []
            templist.append(w1)
            templist.append(w2)
            templist.append(str(int(word_vector[i][j])))
            #print templist
            writer.writerow(templist)
        j = j + 1
    i = i + 1
c.close()